In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from ccai_client import API, File
from ccai_client.file_classes import PathologySlideNode
from histpat_toolkit.image_pyramid import DZIPyramid
from histpat_toolkit.geom import Rectangle
import matplotlib.pyplot as plt
from pathlib import Path
from dataclasses import asdict, dataclass
import json
import shutil
import glob


In [ ]:
# Organization is the codename of your organization in CancerCenter.ai platform -
# usually it is the first part of the site base URL
api = API(organization="platform", save_token_to=".auth.json")

In [ ]:
root_file = File.get_root(api)
my_files = next(
    child for child in root_file.children()
    if child.name.lower() == "my files" or child.name.lower() == "moje pliki"
)

## Upload collage (ISD File)

In [ ]:
@dataclass
class ROI:
    x: float
    y: float
    w: float
    h: float
    color: str = '#E57373' # red
    comment: str | None = None
    order: int = 1
    label: str | bool = True

# represents a rectangle which covers full image
@dataclass
class FullROI:
    color: str = '#94BB66' # green
    comment: str | None = None
    order: int = 0
    label: str | bool = True

# images should have the same length as rois
def make_isd_collage(
    file_name: str,
    parent_path: str | Path,
    images: list[str | Path],
    rois: list[list[ROI | FullROI]],
    magnification: int | None = None,
    mpp: float | None = None,
):
    parent_path = Path(parent_path)
    isd_path = parent_path / f"{file_name}.isd"
    data_folder_path = parent_path / f"{file_name}_data"
    parent_path.mkdir(parents=True, exist_ok=True)

    if data_folder_path.exists():
        print(f"Data folder {data_folder_path} already exists, removing it")
        shutil.rmtree(data_folder_path)

    data_folder_path.mkdir(parents=True, exist_ok=False)

    isd_images = []

    for image_path, rois in zip(images, rois):
        image_path = Path(image_path)
        image_name = image_path.name
        new_image_path = data_folder_path / image_name
        shutil.copy(image_path, new_image_path)
        isd_images.append({
            "type": "single",
            "path": str(new_image_path.relative_to(parent_path)),
            "rois": [asdict(roi) for roi in rois]
        })

    isd_data = {
        "version": "1.1",
        "layout": "wrapped",
        "images": isd_images
    }
    if magnification is not None:
        isd_data["magnification"] = magnification
    if mpp is not None:
        isd_data["mpp"] = mpp

    with open(isd_path, "w") as f:
        f.write(json.dumps(isd_data, indent=4))

    return isd_path, data_folder_path

In [ ]:
images = glob.glob("example_data/collage/*.jpg")
rois = [
    [
        FullROI(comment=f"ROI covering full image {i}", label="Big label"),
        ROI(x=10 * i, y=10 * i, w=500, h=500, comment=f"small ROI in marked in the image {i}", label=False)
    ]
    for i in range(len(images))
]

paths = make_isd_collage("collage", "data", images, rois)
print('Data saved to:', paths)

In [ ]:
slide_id = PathologySlideNode.create_from_files(api, paths, slide_name='collage', parent_file_id=my_files.id)
print('Slide created with id:', slide_id)

In [ ]:
# Note: you may need to wait some time for the slide to be processed
file: PathologySlideNode = File.get(api, slide_id)
dzi_pyramid = DZIPyramid(file.dzi_file)
image = dzi_pyramid.full_image()
plt.imshow(image)

## Upload file from a scanner (SVS example)

In [ ]:
slide_id = PathologySlideNode.create_from_files(api, 'example_data/test.svs', slide_name='test_svs', parent_file_id=my_files.id, verbose=True)
print('Slide created with id:', slide_id)

In [ ]:
# Note: you may need to wait some time for the slide to be processed
slide: PathologySlideNode = File.get(api, slide_id)
dzi_pyramid = DZIPyramid(slide.dzi_file)
image = dzi_pyramid.full_image()
plt.imshow(image)

## Upload a segmentation mask from a file

In [ ]:
from ccai_client.patho import ColorMap

colormap = ColorMap.get_by_codename(api=api, codename="tissue-detection")
color = colormap.colors[0].value

In [ ]:
# Prepare a sample mask (colors mode)

from PIL import Image, ImageDraw, ImageColor

height, width = image.shape[0], image.shape[1]

overlay = Image.new("RGBA", (width, height), (0, 0, 0, 0))
draw = ImageDraw.Draw(overlay)
image_color = ImageColor.getcolor(color, "RGBA")

radius = min(width, height) // 4
center = (width // 2, height // 2)
bbox = [
    center[0] - radius,
    center[1] - radius,
    center[0] + radius,
    center[1] + radius
]
draw.ellipse(bbox, fill=image_color)
overlay.save("data/segmentation.png")

In [ ]:
slide.upload_tiled_mask("data/segmentation.png", color_map="tissue-detection")

In [ ]:
# Prepare a sample mask (keys mode - single channel image with class numbers)
height, width = image.shape[0], image.shape[1]

from PIL import Image, ImageDraw

# Create a zero-filled grayscale image
mask_img = Image.new("L", (width, height), 0)

# Draw a white-filled circle (value 1)
draw = ImageDraw.Draw(mask_img)
radius = min(width, height) // 4
center = (width // 2, height // 2)
bbox = [
    center[0] - radius,
    center[1] - radius,
    center[0] + radius,
    center[1] + radius
]
draw.ellipse(bbox, fill=1)

# Save to PNG
mask_img.save("data/segmentation_keys.png")


In [ ]:
slide.upload_tiled_mask("data/segmentation_keys.png", color_map="tissue-detection", mask_type="KEYS")